In [3]:
import pandas as pd
from config import gkey
import gmaps
import requests

ImportError: cannot import name 'gkey' from 'config' (/Users/Michaelangelo/Documents/Columbia Data Analyst/Python/World_Weather_Analysis/config.py)

In [ ]:
city_data_df = pd.read_csv('weather_data/cities.csv')

In [ ]:
city_data_df.head()

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gkey)

NameError: name 'gmaps' is not defined

In [59]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                 dissipating= False, max_intensity= 300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [64]:
# Percent Humidity
# Assign the locations variable
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
# Figure of heat map
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat map of humidity
heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity = 200, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_map)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [66]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df['Wind Speed']
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed,
                                 dissipating= False, max_intensity= 300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [87]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [88]:
# New dataframe for getting min and max temp
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.count()

City_ID       165
City          165
Country       163
Date          165
Lat           165
Lng           165
Max Temp      165
Humidity      165
Cloudiness    165
Wind Speed    165
dtype: int64

In [89]:
preferred_cities_df = preferred_cities_df.dropna()

In [90]:
preferred_cities_df.count()

City_ID       163
City          163
Country       163
Date          163
Lat           163
Lng           163
Max Temp      163
Humidity      163
Cloudiness    163
Wind Speed    163
dtype: int64

In [2]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

NameError: name 'preferred_cities_df' is not defined

In [98]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print('Hotel not found... skipping')

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [100]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,80.19,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
17,Rikitea,PF,77.79,-23.1203,-134.9692,People ThankYou
20,Kulhudhuffushi,MV,84.22,6.6221,73.0700,Haajy Guest House 1
22,Faya,SA,78.91,18.3851,42.4509,املاك خالد الشعشوع
24,Lhokseumawe,ID,80.73,5.1801,97.1507,OYO 3278 Penginapan Alfajri


In [102]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [103]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [104]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [1]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

NameError: name 'hotel_df' is not defined